<a href="https://colab.research.google.com/github/ekarla/AED2-2022/blob/main/BigData2022_Hadoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## BIG DATA - HADOOP

Projeto realizado na matéria de Pratica em Banco de Dados 2022/1 ☝ <br>
Dupla : Elenkarla Silva e Leide Marina ✌

# Parte 1 - Instalação e Configuração do HADOOP




In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null # Instalando o java 11

In [ ]:
!wget -q https://archive.apache.org/dist/hadoop/common/hadoop-3.3.3/hadoop-3.3.3.tar.gz # Download do Haadoop

In [ ]:
! tar -xzf hadoop-3.3.3.tar.gz #extraindo o arquivo hadoop


gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


In [ ]:
!rm hadoop-3.3.3.tar.gz #removendo o arquivo tar

In [ ]:
!cp -r hadoop-3.3.3/ /usr/local/ #Copiando o hadoop para o diretório /usr/local

In [ ]:
#Exploranddo o diretório hadoop-3.2.3/etc/hadoop 
#!ls /usr/local/hadoop-3.3.3/etc/hadoop

In [ ]:
# configurando variáveis de ambiente
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"
os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.3.3"

In [ ]:
!$HADOOP_HOME/bin/hadoop

Usage: hadoop [OPTIONS] SUBCOMMAND [SUBCOMMAND OPTIONS]
 or    hadoop [OPTIONS] CLASSNAME [CLASSNAME OPTIONS]
  where CLASSNAME is a user-provided Java class

  OPTIONS is none or any of:

buildpaths                       attempt to add class files from build tree
--config dir                     Hadoop config directory
--debug                          turn on shell script debug mode
--help                           usage information
hostnames list[,of,host,names]   hosts to use in slave mode
hosts filename                   list of hosts to use in slave mode
loglevel level                   set the log4j level for this command
workers                          turn on worker mode

  SUBCOMMAND is one of:


    Admin Commands:

daemonlog     get/set the log level for each daemon

    Client Commands:

archive       create a Hadoop archive
checknative   check native Hadoop and compression libraries availability
classpath     prints the class path needed to get the Hadoop jar and the
    

# Inicinado a Parte 2 do trabalho 


In [ ]:
import pandas as pd

In [ ]:
!wget -q https://raw.githubusercontent.com/ekarla/ProjetoBigData/main/shakespeare.txt
!wget -q https://raw.githubusercontent.com/ekarla/ProjetoBigData/main/p2p-Gnutella08.txt

In [ ]:
%%writefile mapper.py

# mapper.py
import sys


class Mapper:

    '''
        O método construtor vai fazer o papel do método initialize.
        O nosso array associativo será um dicionário.
    '''
    def __init__(self):
        self.array_associativo = {}
    
    def mapear(self, doc):
        for linha in doc:
            linha = linha.strip()
            palavras = linha.decode().split()

            for palavra in palavras:
                
                '''
                    Se a palavra já existe como chave no array só incrementar,
                    caso contrário, inicia com 1.
                '''
                if self.array_associativo.get(palavra):
                    self.array_associativo[palavra] +=  1
                else:
                    self.array_associativo[palavra] =  1
    
    def fechar(self):
        for palavra in self.array_associativo.keys():
            print("%s\t%s" % (palavra, self.array_associativo[palavra]))


if __name__ == "__main__":
    mapeador = Mapper()

    mapeador.mapear(sys.stdin.buffer.raw)
    mapeador.fechar()

Writing mapper.py


In [ ]:
%%writefile reducer.py

# reducer.py
from operator import itemgetter
import sys


class Reducer:

    def reduzir(self, contagens):
        palavra_atual = None
        contagem_atual = 0
        palavra = None

        for linha in contagens:
            linha = linha.strip()
            palavra, qtd = linha.decode().split('\t', 1)

            try:
                qtd = int(qtd)
            except ValueError:
                continue
            
            if palavra_atual == palavra:
                contagem_atual += qtd
            else:
                if palavra_atual:
                    print('%s\t%s' % (palavra_atual, contagem_atual))

                contagem_atual = qtd
                palavra_atual = palavra


if __name__ == "__main__":
    reduzidor = Reducer()

    reduzidor.reduzir(sys.stdin.buffer.raw)

Writing reducer.py


In [ ]:
!chmod u+rwx /content/mapper.py
!chmod u+rwx /content/reducer.py

In [ ]:
!rm -rf saidas/

**Documento** **Shakespare**

In [ ]:
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-3.3.3.jar -input /content/shakespeare.txt -output /content/saidas/output-shakespeare -file /content/mapper.py  -file /content/reducer.py  -mapper 'python mapper.py'  -reducer 'python reducer.py'

2022-11-14 13:36:18,385 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/content/mapper.py, /content/reducer.py] [] /tmp/streamjob4928222347003618207.jar tmpDir=null
2022-11-14 13:36:19,221 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-11-14 13:36:19,439 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-11-14 13:36:19,439 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2022-11-14 13:36:19,460 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2022-11-14 13:36:19,659 INFO mapred.FileInputFormat: Total input files to process : 1
2022-11-14 13:36:19,681 INFO mapreduce.JobSubmitter: number of splits:1
2022-11-14 13:36:20,129 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local525844481_0001
2022-11-14 13:36:20,129 INFO mapreduce.JobSubmitter: Executing with tokens: []
2022-11-14 13:36:20,471 INFO mapred.Loca

Documento Gnutella

In [ ]:
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-3.3.3.jar -input /content/p2p-Gnutella08.txt -output /content/saidas/output-Gnutella -file /content/mapper.py  -file /content/reducer.py  -mapper 'python mapper.py'  -reducer 'python reducer.py'

2022-11-14 13:36:23,826 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/content/mapper.py, /content/reducer.py] [] /tmp/streamjob6360441805324245754.jar tmpDir=null
2022-11-14 13:36:24,527 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-11-14 13:36:24,674 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-11-14 13:36:24,674 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2022-11-14 13:36:24,699 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2022-11-14 13:36:24,853 INFO mapred.FileInputFormat: Total input files to process : 1
2022-11-14 13:36:24,879 INFO mapreduce.JobSubmitter: number of splits:1
2022-11-14 13:36:25,130 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local217091535_0001
2022-11-14 13:36:25,130 INFO mapreduce.JobSubmitter: Executing with tokens: []
2022-11-14 13:36:25,480 INFO mapred.Loca

# Respondendo as questões Versão 1 ✅

## Resposta documeto Shakespare

Q_1 -Contando a quantidade de palvras distintas.

In [ ]:
!cat /content/saidas/output-shakespeare/part-00000 | wc -l

5662


Q_2 - Listando as 10 palavras mais ocorridas.

In [ ]:
!cat /content/saidas/output-shakespeare/part-00000 | sort -k 2 -n -r | head 

the	513
of	456
to	401
my	341
in	338
I	327
and	316
thy	252
that	248
And	239


Q_3 - Histograma das palavras.

In [ ]:
!cat /content/saidas/output-shakespeare/part-00000 | sort -k 2 -n -r  > /content/fileShak.txt

In [ ]:
arq_shak = pd.read_csv('/content/fileShak.txt', sep = '\t', header=None)
arq_shak.columns = ["word",'count']
bin =  int(int(arq_shak['count'].unique().size)/10)
histogramaS = pd.cut(arq_shak['count'],bin,include_lowest=True).value_counts()
histogramaS

(0.487, 57.889]       4669
(57.889, 114.778]       26
(114.778, 171.667]       6
(171.667, 228.556]       4
(285.444, 342.333]       4
(228.556, 285.444]       3
(399.222, 456.111]       2
(456.111, 513.0]         1
(342.333, 399.222]       0
Name: count, dtype: int64

## Resposta documento Gnutella

Quantidade de palavras distintas


In [ ]:
!cat /content/saidas/output-Gnutella/part-00000 | wc -l

6323


As 10 palavras mais ocorridas

In [ ]:
!cat /content/saidas/output-Gnutella/part-00000 | sort -k 2 -n -r | head

123	97
127	95
367	94
424	92
427	91
266	91
264	91
251	91
249	90
145	90


Histograma textual

In [ ]:
!cat /content/saidas/output-shakespeare/part-00000 | sort -k 2 -n -r  > /content/Gnutella.txt

In [ ]:
arq_gnutella = pd.read_csv('/content/Gnutella.txt', sep = '\t', header=None) #lendo arquivo txt
arq_gnutella.columns = ["word",'count']  #criando columns
bin =  int(int(arq_gnutella['count'].unique().size)/10)
histogramaG = pd.cut(arq_gnutella['count'],bin,include_lowest=True).value_counts()
histogramaG

(0.487, 57.889]       4669
(57.889, 114.778]       26
(114.778, 171.667]       6
(171.667, 228.556]       4
(285.444, 342.333]       4
(228.556, 285.444]       3
(399.222, 456.111]       2
(456.111, 513.0]         1
(342.333, 399.222]       0
Name: count, dtype: int64

# Respondendo as questões Versão 2 ⚡

Nesta versão utilizamos uma Regex para pegarmos somente palavras.<br>
Ignorando os caracteres especiais.

In [ ]:
%%writefile mapper_regex.py

# mapper_regex.py
import re
import sys

'''
  Construimoa um metodo que serve como o initializae
  O array associativo será um dicionário.
'''

class Mapper:

   
    def __init__(self):
        self.array_associativo = {}
    
    def mapear(self, doc):
        for linha in doc:
            linha = linha.strip()
            palavras = linha.decode().split()

            for palavra in palavras:

                '''
                    Ao percorre o loop aplicamos a regex na palavra para verificar  
                    se existem apenas letras.
                '''
                if re.match(r"^[a-zA-Z]+$", str(palavra)):

                    '''
                        Se a palavra já existe como chave no array então incrementamos,
                        caso contrário, inicia com 1.
                    '''
                    if self.array_associativo.get(palavra):
                        self.array_associativo[palavra] +=  1
                    else:
                        self.array_associativo[palavra] =  1
    
    def fechar(self):
        for palavra in self.array_associativo.keys():
            print("%s\t%s" % (palavra, self.array_associativo[palavra]))


if __name__ == "__main__":
    mapeador = Mapper()

    mapeador.mapear(sys.stdin.buffer.raw)
    mapeador.fechar()

Overwriting mapper_regex.py


Dando permissão ao mapper_regex

In [ ]:
!chmod u+rwx /content/mapper_regex.py

In [ ]:
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-3.3.3.jar -input /content/shakespeare.txt -output /content/saidas/output-shakespeare-regex -file /content/mapper_regex.py  -file /content/reducer.py  -mapper 'python mapper_regex.py'  -reducer 'python reducer.py'

2022-11-14 13:46:36,506 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/content/mapper_regex.py, /content/reducer.py] [] /tmp/streamjob13644109895896143360.jar tmpDir=null
2022-11-14 13:46:37,316 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-11-14 13:46:37,474 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-11-14 13:46:37,474 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2022-11-14 13:46:37,501 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2022-11-14 13:46:37,513 ERROR streaming.StreamJob: Error Launching job : Output directory file:/content/saidas/output-shakespeare-regex already exists
Streaming Command Failed!


# Respodendo as perguntas utilizando a versão 2 ✔

### Quantidade de palavras distintas

In [ ]:
!cat /content/saidas/output-shakespeare-regex/part-00000 | wc -l

3224


### As 10 palavras que mais ocorreram

In [ ]:
!cat /content/saidas/output-shakespeare-regex/part-00000 | sort -k 2 -n -r | head 

the	513
of	456
to	401
my	341
in	338
I	327
and	316
thy	252
that	248
And	239


In [ ]:
!cat /content/saidas/output-shakespeare-regex/part-00000 | sort -k 2 -n -r  > /content/fileShakRegex.txt

In [ ]:
arq_shak_regex = pd.read_csv('/content/fileShakRegex.txt', sep = '\t', header=None)
arq_shak_regex.columns = ["word",'count']
bin =  int(int(arq_shak_regex['count'].unique().size)/10)
histogramaSR = pd.cut(arq_shak_regex['count'],bin,include_lowest=True).value_counts()
histogramaSR


(0.487, 57.889]       3178
(57.889, 114.778]       26
(114.778, 171.667]       6
(171.667, 228.556]       4
(285.444, 342.333]       4
(228.556, 285.444]       3
(399.222, 456.111]       2
(456.111, 513.0]         1
(342.333, 399.222]       0
Name: count, dtype: int64

## Arquivo Gnutellla

In [ ]:
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-3.3.3.jar -input /content/p2p-Gnutella08.txt -output /content/saidas/output-Gnutella-regex -file /content/mapper_regex.py  -file /content/reducer.py  -mapper 'python mapper_regex.py'  -reducer 'python reducer.py'

2022-11-14 13:46:39,255 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/content/mapper_regex.py, /content/reducer.py] [] /tmp/streamjob7017650700077531535.jar tmpDir=null
2022-11-14 13:46:40,063 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-11-14 13:46:40,251 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-11-14 13:46:40,251 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2022-11-14 13:46:40,277 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2022-11-14 13:46:40,290 ERROR streaming.StreamJob: Error Launching job : Output directory file:/content/saidas/output-Gnutella-regex already exists
Streaming Command Failed!


### Quantidade de palavras distintas.

In [ ]:
!cat /content/saidas/output-Gnutella-regex/part-00000 | wc -l 

13


### As 10 palavras que mais ocorreram.

In [ ]:
!cat /content/saidas/output-Gnutella-regex/part-00000 | sort -k 2 -n -r | head 

Directed	2
ToNodeId	1
saved	1
pair	1
of	1
nodes	1
network	1
is	1
graph	1
Gnutella	1


### Histograma Textual

In [ ]:
!cat /content/saidas/output-Gnutella-regex/part-00000 | sort -k 2 -n -r  > /content/fileGnutelaReg.txt

In [ ]:
arq_Gnutella_regex = pd.read_csv('/content/fileGnutelaReg.txt', sep = '\t', header=None)
arq_Gnutella_regex.columns = ["word",'count']
bin =  int(arq_Gnutella_regex['count'].unique().size)
histogramaGR = pd.cut(arq_shak_regex['count'],bin,include_lowest=True).value_counts()
histogramaGR

(0.487, 257.0]    3217
(257.0, 513.0]       7
Name: count, dtype: int64

In [ ]:
!cat /content/fileShak.txt 

the	513
of	456
to	401
my	341
in	338
I	327
and	316
thy	252
that	248
And	239
with	209
a	202
thou	196
is	172
not	164
you	134
be	128
this	121
for	119
all	118
me	111
or	110
love	103
his	102
by	100
it	98
your	96
thee	94
The	91
do	91
as	89
But	84
To	82
That	82
doth	82
on	81
from	81
are	80
so	79
Project	78
but	76
For	73
have	69
which	67
no	65
their	61
When	57
what	57
Gutenberg-tm	53
shall	51
can	51
will	50
when	50
If	48
more	47
should	46
beauty	45
sweet	44
Which	43
So	43
mine	43
they	41
thee,	40
her	40
O!	39
may	39
love,	39
any	39
As	38
than	37
In	37
if	37
he	37
hath	37
make	36
art	36
work	35
then	35
eyes	35
at	35
self	32
Or	32
him	32
am	32
me,	31
heart	31
fair	31
she	30
My	30
must	30
every	30
thine	29
such	29
yet	28
where	28
Thy	28
Then	28
own	28
nor	28
eye	28
see	27
other	27
now	27
electronic	27
being	27
time	26
still	26
like	26
dost	26
did	26
works	25
was	25
true	25
Thou	25
world	24
those	24
some	24
Nor	24
most	24
were	23
upon	23
one	23
how	23
though	22
we	21
Of	21
How	21
full	21
would	20
t

In [ ]:
!cat /content/saidas/output-shakespeare-regex/part-00000 | egrep -o [0-9]+ | sort -n  | uniq -c 

   1909 1
    482 2
    252 3
    105 4
     81 5
     62 6
     45 7
     31 8
     12 9
     28 10
     14 11
     18 12
     17 13
      9 14
      9 15
      9 16
      4 17
      6 18
      2 19
      6 20
      4 21
      1 22
      4 23
      5 24
      4 25
      5 26
      5 27
      7 28
      2 29
      4 30
      2 31
      4 32
      4 35
      2 36
      5 37
      1 38
      2 39
      1 40
      1 41
      3 43
      1 44
      1 45
      1 46
      1 47
      1 48
      2 50
      2 51
      2 57
      1 61
      1 65
      1 67
      1 69
      1 73
      1 76
      1 78
      1 79
      1 80
      2 81
      3 82
      1 84
      1 89
      2 91
      1 94
      1 96
      1 98
      1 100
      1 102
      1 103
      1 110
      1 111
      1 118
      1 119
      1 121
      1 128
      1 134
      1 164
      1 172
      1 196
      1 202
      1 209
      1 239
      1 248
      1 252
      1 316
      1 327
      1 338
      1 341
      1 401
      1 456
      

In [ ]:
!cat /content/saidas/output-shakespeare-regex/part-00000 | egrep -o [0-9]+ | sort -n  | uniq -c | head -n 9| awk '{SUM += $1} END {print "1 --- 10: " SUM}'

1 --- 10: 2979
